In [2]:
from src.acoustic_analyser import structure

test_structure = structure()

m1, _ = test_structure.add_member(length=0.5, rho=1, cross_section_area=1, youngs_modulus=1, inertia=1)
m2, _ = test_structure.add_member(length=0.5, rho=1, cross_section_area=1, youngs_modulus=1, inertia=1)
m3, _ = test_structure.add_member(length=1, rho=1, cross_section_area=1, youngs_modulus=1, inertia=1)

j, _ = test_structure.fixed_end(member=m1)
j1,_ =test_structure.two_member_joint(member_1=m1, member_2=m2, theta=0)
j2,_ =test_structure.two_member_joint(member_1=m2, member_2=m3, theta=0)

j3, _ = test_structure.free_end(member=m3)

lhs=test_structure.get_equation_matrix()
lhs

[-A-1 - I*C-1*exp(-2.0*I*sqrt(w)*x) + D-1*(-exp(-sqrt(w)*x)*exp(-I*sqrt(w)*x) + I*exp(-sqrt(w)*x)*exp(-I*sqrt(w)*x)),
 -B-1 + C-1*(-exp(-sqrt(w)*x)*exp(-I*sqrt(w)*x) - I*exp(-sqrt(w)*x)*exp(-I*sqrt(w)*x)) + I*D-1*exp(-2.0*sqrt(w)*x),
 -C-1,
 -D-1,
 A-1 - A0,
 B-1 - B0,
 -C0,
 -D0,
 A0 - A1,
 B0 - B1,
 -I*A1*exp(-2.0*I*sqrt(w)*x) + B1*(exp(-sqrt(w)*x)*exp(-I*sqrt(w)*x) - I*exp(-sqrt(w)*x)*exp(-I*sqrt(w)*x)) - C1,
 A1*(exp(-sqrt(w)*x)*exp(-I*sqrt(w)*x) + I*exp(-sqrt(w)*x)*exp(-I*sqrt(w)*x)) + I*B1*exp(-2.0*sqrt(w)*x) - D1]

In [7]:
j.reflection_matrix

Matrix([
[                                                -I*exp(-2.0*I*sqrt(w)*x), -exp(-sqrt(w)*x)*exp(-I*sqrt(w)*x) + I*exp(-sqrt(w)*x)*exp(-I*sqrt(w)*x)],
[-exp(-sqrt(w)*x)*exp(-I*sqrt(w)*x) - I*exp(-sqrt(w)*x)*exp(-I*sqrt(w)*x),                                                    I*exp(-2.0*sqrt(w)*x)]])